In [1]:
import pandas as pd
import numpy as np
import re
import time

##Bs4 para o parsing
import bs4 as bs4
import requests as rq
import json

# Coleta dos dados

### Definimos as buscas e a página (url)

In [2]:
buscas = ["algoritmos+geneticos", "machine+learning"]
url = "https://scholar.google.com.br/scholar?start={itens}&q={busca}&hl=pt-BR&as_sdt=0,5"

### Cada página do Google Scholar aprensenta 9 itens por página. Na url, o argumento 'start=' refere-se ao item que será mostrado primeiro. Portanto, para buscar em mais de uma página, devemos iterar de 10 em 10.

In [8]:
for busca in buscas:
    # de 0 1 50 de 10 em 10
    for itens in range(0, 200, 10):
        ## Definindo as urls de busca
        urll = url.format(busca=busca, itens=itens)
        print(urll)
        ## Passamos a url formatada para o request
        ## O objeto response terá as informações da página, o código fonte
        response = rq.get(urll)
        
        ## Salvamos os dados brutos em arquivos html
        with open("./dados_brutos/{}_{}.html".format(busca, itens), 'w+') as output:
            ## Código fonte no formato string
            output.write(response.text)
        ## Dois segundos entre cada requisição
        time.sleep(2)

https://scholar.google.com.br/scholar?start=0&q=algoritmos+geneticos&hl=pt-BR&as_sdt=0,5
https://scholar.google.com.br/scholar?start=10&q=algoritmos+geneticos&hl=pt-BR&as_sdt=0,5
https://scholar.google.com.br/scholar?start=20&q=algoritmos+geneticos&hl=pt-BR&as_sdt=0,5
https://scholar.google.com.br/scholar?start=30&q=algoritmos+geneticos&hl=pt-BR&as_sdt=0,5
https://scholar.google.com.br/scholar?start=40&q=algoritmos+geneticos&hl=pt-BR&as_sdt=0,5
https://scholar.google.com.br/scholar?start=50&q=algoritmos+geneticos&hl=pt-BR&as_sdt=0,5
https://scholar.google.com.br/scholar?start=60&q=algoritmos+geneticos&hl=pt-BR&as_sdt=0,5
https://scholar.google.com.br/scholar?start=70&q=algoritmos+geneticos&hl=pt-BR&as_sdt=0,5
https://scholar.google.com.br/scholar?start=80&q=algoritmos+geneticos&hl=pt-BR&as_sdt=0,5
https://scholar.google.com.br/scholar?start=90&q=algoritmos+geneticos&hl=pt-BR&as_sdt=0,5
https://scholar.google.com.br/scholar?start=100&q=algoritmos+geneticos&hl=pt-BR&as_sdt=0,5
https://sc

# Processando os dados brutos

In [42]:
## Vamos abrir os htmls salvos para processá-los
for busca in buscas:
        for itens in range(0,200,10):
            ## Abre e lê cada página
            with open("./dados_brutos/{}_{}.html".format(busca, itens), 'r+') as inp:
                page_html = inp.read()

                parsed = bs4.BeautifulSoup(page_html)

                data = {}

                ##Todos os elementos que precisamos estão dentro de uma div com class="gs_ri"
                ##Para cada div dessa na página vamos procurar pelos elementos
                main_tag = parsed.find_all("div", attrs={"class": "gs_ri"})
                
                for e in main_tag:
                    ##Link e título
                    lista_links = e.h3.findAll('a')
                    for i in lista_links:
                        link = i['href']
                        titulo = i.text
                    
                    #Número de citações
                    lista_citacoes = e.find_all("a", attrs={"href":re.compile(r"cites")})
                    for i in lista_citacoes:
                        citacoes = re.findall(r'[0-9]+', i.text)
                    
                    ##Ano de publicação
                    lista_anos = e.find_all("div", attrs={"class": "gs_a"})
                    for i in lista_anos:
                        ano = re.findall(r'[0-9]+', i.text)
                
                    ##Tipo do arquivo
                    lista_tipo_arquivo = e.find_all("span", attrs={"class": "gs_ct1"})
                    for i in lista_tipo_arquivo:
                        tipo_arquivo = i.text
                    
                    ##Quantas versões do arquivo
                    lista_versoes = e.find_all("a", attrs={"href":re.compile(r"cluster")})
                    for i in lista_versoes:
                        versao = re.findall(r'[0-9]+', i.text)
                    
                    ## vamos salvar os itens em um arquivo json contendo todas as informações
                    with open("parsed_itens.json", 'a+') as output:
                        ## dicionário que será convertido em json
                        data = {"link": link, "titulo": titulo, "citacoes": citacoes,
                                "ano": ano, "tipo_arquivo": tipo_arquivo, "versao": versao, "busca": busca}
                        output.write("{}\n".format(json.dumps(data)))

In [43]:
df = pd.read_json("parsed_itens.json", lines=True)

In [44]:
df.head()

,link,titulo,citacoes,ano,tipo_arquivo,versao,busca
0,https://www.academia.edu/download/55091016/alg...,Introducción a los algoritmos genéticos y la p...,[75],[2010],[LIVRO],[3],algoritmos+geneticos
1,https://www.researchgate.net/profile/Pablo_Est...,Optimización mediante algoritmos genéticos,[51],[1997],[PDF],[2],algoritmos+geneticos
2,http://it.uc3m.es/jvillena/irc/practicas/06-07...,Algoritmos genéticos,[18],"[2007, 3]",[PDF],[5],algoritmos+geneticos
3,http://cursos.itam.mx/akuri/PUBLICA.CNS/2000/A...,Algoritmos genéticos,[26],[2002],[LIVRO],[2],algoritmos+geneticos
4,https://the-geek.org/docs/algen/algen.html,Algoritmos genéticos y computación evolutiva,[29],[2004],[HTML],[2],algoritmos+geneticos


In [47]:
df['link']

0      https://www.academia.edu/download/55091016/alg...
1      https://www.researchgate.net/profile/Pablo_Est...
2      http://it.uc3m.es/jvillena/irc/practicas/06-07...
3      http://cursos.itam.mx/akuri/PUBLICA.CNS/2000/A...
4             https://the-geek.org/docs/algen/algen.html
                             ...                        
395    https://www.nature.com/articles/s41586-018-0337-2
396    https://ieeexplore.ieee.org/abstract/document/...
397    https://books.google.com.br/books?hl=pt-BR&lr=...
398                     https://arxiv.org/abs/1605.07277
399    http://papers.nips.cc/paper/2571-using-machine...
Name: link, Length: 400, dtype: object

### Necessário converter para string, pois o feather não aceita o formato em questão

In [31]:
df.dtypes

link            object
titulo          object
citacoes        object
ano             object
tipo_arquivo    object
versao          object
busca           object
dtype: object

In [46]:
df['link'] = df['link'].astype(str)
df['titulo'] = df['titulo'].astype(str)
df['citacoes'] = df['citacoes'].astype(str)
df['ano'] = df['ano'].astype(str)
df['tipo_arquivo'] = df['tipo_arquivo'].astype(str)
df['versao'] = df['versao'].astype(str)
df['busca'] = df['busca'].astype(str)

In [37]:
df.shape

(400, 7)

In [18]:
df.isnull().sum()

link            0
titulo          0
citacoes        0
ano             0
tipo_arquivo    0
versao          0
busca           0
dtype: int64

### Exportando o DataFrame em formato feather

In [36]:
df.to_feather("raw_data.feather")

### O labeling será feito manualmente, portano exportamos o DataFrame para um csv

In [48]:
df.to_csv("raw_data_sem_labels.csv")